In [ ]:
%matplotlib inline

# Redes neuronales

Las redes neuronales se pueden construir usando el paquete ``torch.nn``.

Ahora que tuvo una idea de ``autograd``, ``nn`` depende de
``autograd`` para definir modelos y diferenciarlos.
Un ``nn.Module`` contiene capas y un método ``forward(input)`` que
devuelve la ``salida``.

Por ejemplo, mira esta red que clasifica imágenes de dígitos:

![](./assets/lenet.png)

Se trata de una simple red feed-forward. Toma la entrada, la alimenta
a través de varias capas una tras otra, y finalmente genera una predicción.

Un procedimiento de entrenamiento típico para una red neuronal es el siguiente:

- Definir la red neuronal que tiene algunos parámetros aprendibles (o
  pesos)
- Iterar sobre un conjunto de datos de entradas
- Proceso de entrada a través de la red
- Calcule la pérdida (qué tan lejos está la salida de ser correcta)
- Propaga los gradientes de vuelta a los parámetros de la red
- Actualice los pesos de la red, generalmente usando una regla de actualización simple:
  ``peso = peso - tasa_de_aprendizaje * gradiente``

## Definir la red

Definamos esta red:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Solo tienes que definir la función ``forward``, y la función ``backward``
(donde se calculan los gradientes) se define automáticamente usando ``autograd``.
Puede usar cualquiera de las operaciones de Tensor en la función ``forward``.

Los parámetros que se pueden aprender de un modelo son devueltos por ``net.parameters()``



In [ ]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

Probemos una entrada aleatoria de 32x32.
Nota: el tamaño de entrada esperado de esta red (LeNet) es 32x32. Para usar esta red en
el conjunto de datos MNIST, cambie el tamaño de las imágenes del conjunto de datos a 32x32.



In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

Ponemos en cero los búferes de gradiente de todos los parámetros y backprops con gradientes aleatorios:


In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>Nota:</h4>
<p>

``torch.nn`` solo admite minilotes. Todo el paquete ``torch.nn`` solo admite entradas que son un mini lote de muestras, y no una sola muestra.
Por ejemplo, ``nn.Conv2d`` tomará un tensor 4D de ``nMuestras x nCanales x Alto x Ancho``.
Si tiene una sola muestra, simplemente use ``input.unsqueeze(0)`` para agregar
    una dimensión de lote falsa.
</p></div>

**En este punto, cubrimos:**
  - Definición de una red neuronal
  - Procesamiento de entradas y llamadas hacia atrás.

**Aún quedan:**
  - Cálculo de la pérdida
  - Actualización de los pesos de la red.

## Función de pérdida
Una función de pérdida toma el par de entradas (salida, objetivo) y calcula una
valor que estima qué tan lejos está la salida del objetivo.

Hay varios diferentes [funciones de pérdida](https://pytorch.org/docs/nn.html#loss-functions) bajo el
paquete ``nn``.
Una pérdida simple es: ``nn.MSELoss`` que calcula el error cuadrático medio
entre la salida y el destino.

Por ejemplo:

In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Ahora, si sigues ``loss`` en dirección hacia atrás, usando su
atributo ``.grad_fn``, verá un gráfico de cálculos que parece
como esto:

:: 

    entrada -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
           -> aplanar -> lineal -> relu -> lineal -> relu -> lineal
           -> Pérdida de MSEL
           -> pérdida

Entonces, cuando llamamos a ``loss.backward()``, todo el gráfico se diferencia
w.r.t. los parámetros de la red neuronal y todos los tensores en el gráfico que tienen
``requires_grad=True`` tendrá su Tensor ``.grad`` acumulado con el gradiente.

A modo de ilustración, sigamos unos pasos hacia atrás:

In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop
Para retropropagar el error todo lo que tenemos que hacer es ``loss.backward()``.
Sin embargo, debe borrar los gradientes existentes, de lo contrario, los gradientes serán
acumulados a los gradientes existentes.


Ahora llamaremos a ``loss.backward()``, y echaremos un vistazo al sesgo de conv1
gradientes antes y después del retroceso.



In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

Ahora, hemos visto cómo usar las funciones de pérdida.

**Leer más tarde:**

   El paquete de red neuronal contiene varios módulos y funciones de pérdida
   que forman los componentes básicos de las redes neuronales profundas. Una lista completa con
   la documentación está [aquí](https://pytorch.org/docs/nn).

**Lo único que queda por aprender es:**

   - Actualización de los pesos de la red.

## Actualizar los pesos
La regla de actualización más simple utilizada en la práctica es el Descenso de gradiente estocástico (SGD):

``peso = peso - tasa_de_aprendizaje * gradiente``

Podemos implementar esto usando código Python simple:

```python

     tasa_de_aprendizaje = 0.01
     for f in net.parameters():
         f.data.sub_(f.grad.data * tasa_de_aprendizaje)
```

Sin embargo, a medida que usa redes neuronales, desea usar varios
actualizar reglas como SGD, Nesterov-SGD, Adam, RMSProp, etc.
Para habilitar esto, existe un pequeño paquete: ``torch.optim`` que
implementa todos estos métodos. Usarlo es muy simple:

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update